<a href="https://colab.research.google.com/github/Tonry12/Data_joyx2/blob/main/Lottery_Prediction_model_LSTM_(-1)(100)(0.01).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PrepareDATA

Link: https://medium.com/@polanitzer/forecasting-the-next-winning-numbers-in-the-texas-lottery-mega-millions-drawing-using-a-deep-4c37ee6d30f0

In [706]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [707]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os
import pickle

In [708]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns

In [709]:
lotto = pd.read_csv('/content/drive/MyDrive/SPN/Predictive lottery.xlsx - since 30 years ago.csv')

In [710]:
lotto = lotto.drop(columns=['Date','Unnamed: 1','Unnamed: 2','Unnamed: 3'])

In [711]:
lotto

,DATE,รางวัลที่ 1,2 ตัวบน,3 ตัวบน,2 ตัวล่าง,3 ตัวหน้า,3 ตัวหน้า.1,3 ตัวล่าง,3 ตัวล่าง.1,weekday_dum,weekday
0,5/16/2024,205690,90,690,60,747,885,70,137,7,Saturday
1,5/2/2024,980116,16,116,17,104,763,634,833,4,Wednesday
2,4/16/2024,943598,98,598,79,727,729,154,200,3,Tuesday
3,4/1/2024,803481,81,481,90,122,809,559,947,2,Monday
4,3/16/2024,997626,26,626,78,509,571,329,794,7,Saturday
...,...,...,...,...,...,...,...,...,...,...,...
717,5/16/1994,2071764,64,764,29,93,544,535,746,2,Monday
718,5/2/1994,3072877,77,877,69,535,317,170,521,2,Monday
719,4/16/1994,7893913,13,913,21,785,982,151,594,7,Saturday
720,4/1/1994,4269815,15,815,29,675,35,704,868,6,Friday


In [712]:
lotto.rename(columns={'DATE': 'Date','รางวัลที่ 1': '1st','3 ตัวบน':'top3','2 ตัวล่าง':'up2'},inplace=True)

In [713]:
lotto

,Date,1st,2 ตัวบน,top3,up2,3 ตัวหน้า,3 ตัวหน้า.1,3 ตัวล่าง,3 ตัวล่าง.1,weekday_dum,weekday
0,5/16/2024,205690,90,690,60,747,885,70,137,7,Saturday
1,5/2/2024,980116,16,116,17,104,763,634,833,4,Wednesday
2,4/16/2024,943598,98,598,79,727,729,154,200,3,Tuesday
3,4/1/2024,803481,81,481,90,122,809,559,947,2,Monday
4,3/16/2024,997626,26,626,78,509,571,329,794,7,Saturday
...,...,...,...,...,...,...,...,...,...,...,...
717,5/16/1994,2071764,64,764,29,93,544,535,746,2,Monday
718,5/2/1994,3072877,77,877,69,535,317,170,521,2,Monday
719,4/16/1994,7893913,13,913,21,785,982,151,594,7,Saturday
720,4/1/1994,4269815,15,815,29,675,35,704,868,6,Friday


In [714]:
ls= lotto.iloc[:,9:11]

In [715]:
lu= lotto.iloc[:,0:5]

In [716]:
lt = pd.concat([lu, ls], axis=1)

In [717]:
lt

,Date,1st,2 ตัวบน,top3,up2,weekday_dum,weekday
0,5/16/2024,205690,90,690,60,7,Saturday
1,5/2/2024,980116,16,116,17,4,Wednesday
2,4/16/2024,943598,98,598,79,3,Tuesday
3,4/1/2024,803481,81,481,90,2,Monday
4,3/16/2024,997626,26,626,78,7,Saturday
...,...,...,...,...,...,...,...
717,5/16/1994,2071764,64,764,29,2,Monday
718,5/2/1994,3072877,77,877,69,2,Monday
719,4/16/1994,7893913,13,913,21,7,Saturday
720,4/1/1994,4269815,15,815,29,6,Friday


In [718]:
lt = lt.drop(columns=['2 ตัวบน'])

In [719]:
more = lt[lt['1st'] >= 999999]
more.count()

Date           20
1st            20
top3           20
up2            20
weekday_dum    20
weekday        20
dtype: int64

In [720]:
lt = lt[lt['1st'] <= 999999]

In [721]:
lt['up2'] = lt['up2'].apply(lambda x: '{:02}'.format(x))

In [722]:
lt['top3'] = lt['top3'].apply(lambda x: '{:03}'.format(x))

In [723]:
lt['Date'] = pd.to_datetime(lt['Date'])

In [724]:
lt['Year'] = pd.to_datetime(lt['Date']).dt.year
lt['Month'] = pd.to_datetime(lt['Date']).dt.month
lt['Day'] = pd.to_datetime(lt['Date']).dt.day

In [725]:
lt_1 = lt.copy()

In [726]:
lt_1['up2'] = lt_1['up2'].astype(str)

# Split each value in the 'digi' column into its individual digits
lt_1['Tens digit'] = lt_1['up2'].apply(lambda x: int(x[0]))
lt_1['Units digit'] = lt_1['up2'].apply(lambda x: int(x[1]) if len(x) > 1 else 0)

In [727]:
lt_1['top3'] = lt_1['top3'].astype(str)

# Split each value in the 'digi' column into its individual digits
lt_1['c3_r1'] = lt_1['top3'].apply(lambda x: int(x[0]))
lt_1['c3_r2'] = lt_1['top3'].apply(lambda x: int(x[1]))
lt_1['c3_r3'] = lt_1['top3'].apply(lambda x: int(x[2]))

In [728]:
lt_2 = lt_1.copy()

# LSTM Model

In [834]:
lt_2

,Date,1st,top3,up2,weekday_dum,weekday,Year,Month,Day,Tens digit,Units digit,c3_r1,c3_r2,c3_r3
0,2024-05-16,205690,690,60,7,Saturday,2024,5,16,6,0,6,9,0
1,2024-05-02,980116,116,17,4,Wednesday,2024,5,2,1,7,1,1,6
2,2024-04-16,943598,598,79,3,Tuesday,2024,4,16,7,9,5,9,8
3,2024-04-01,803481,481,90,2,Monday,2024,4,1,9,0,4,8,1
4,2024-03-16,997626,626,78,7,Saturday,2024,3,16,7,8,6,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,1995-03-16,646847,847,67,5,Thursday,1995,3,16,6,7,8,4,7
698,1995-03-01,411454,454,71,4,Wednesday,1995,3,1,7,1,4,5,4
699,1995-02-16,869070,070,95,5,Thursday,1995,2,16,9,5,0,7,0
700,1995-02-01,198162,162,48,4,Wednesday,1995,2,1,4,8,1,6,2


In [835]:
sh = lt_2.shape

In [836]:
z = 2

In [837]:
lt_3 = lt_2.iloc[z:sh[0], 0:sh[1]]

In [838]:
lt_3

,Date,1st,top3,up2,weekday_dum,weekday,Year,Month,Day,Tens digit,Units digit,c3_r1,c3_r2,c3_r3
2,2024-04-16,943598,598,79,3,Tuesday,2024,4,16,7,9,5,9,8
3,2024-04-01,803481,481,90,2,Monday,2024,4,1,9,0,4,8,1
4,2024-03-16,997626,626,78,7,Saturday,2024,3,16,7,8,6,2,6
5,2024-03-01,253603,603,79,6,Friday,2024,3,1,7,9,6,0,3
6,2024-02-16,941395,395,43,6,Friday,2024,2,16,4,3,3,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,1995-03-16,646847,847,67,5,Thursday,1995,3,16,6,7,8,4,7
698,1995-03-01,411454,454,71,4,Wednesday,1995,3,1,7,1,4,5,4
699,1995-02-16,869070,070,95,5,Thursday,1995,2,16,9,5,0,7,0
700,1995-02-01,198162,162,48,4,Wednesday,1995,2,1,4,8,1,6,2


## ไม่ล่วงหน้า

In [839]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [840]:
dfp = lt_3.copy()

In [841]:
df = dfp.reset_index()

In [842]:
df

,index,Date,1st,top3,up2,weekday_dum,weekday,Year,Month,Day,Tens digit,Units digit,c3_r1,c3_r2,c3_r3
0,2,2024-04-16,943598,598,79,3,Tuesday,2024,4,16,7,9,5,9,8
1,3,2024-04-01,803481,481,90,2,Monday,2024,4,1,9,0,4,8,1
2,4,2024-03-16,997626,626,78,7,Saturday,2024,3,16,7,8,6,2,6
3,5,2024-03-01,253603,603,79,6,Friday,2024,3,1,7,9,6,0,3
4,6,2024-02-16,941395,395,43,6,Friday,2024,2,16,4,3,3,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,697,1995-03-16,646847,847,67,5,Thursday,1995,3,16,6,7,8,4,7
696,698,1995-03-01,411454,454,71,4,Wednesday,1995,3,1,7,1,4,5,4
697,699,1995-02-16,869070,070,95,5,Thursday,1995,2,16,9,5,0,7,0
698,700,1995-02-01,198162,162,48,4,Wednesday,1995,2,1,4,8,1,6,2


In [843]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [844]:
df2 = df.copy()

In [845]:
df1 = df.copy()
df.drop(['index','Date','1st','weekday_dum','weekday','Year','Month','Day','Tens digit','Units digit','c3_r1','c3_r2','c3_r3'], axis=1, inplace=True)

number_of_features = df.shape[1]

In [846]:
df

,top3,up2
0,598,79
1,481,90
2,626,78
3,603,79
4,395,43
...,...,...
695,847,67
696,454,71
697,070,95
698,162,48


In [847]:
df1.drop(['index','1st','weekday_dum','weekday','Year','Month','Day','Tens digit','Units digit','c3_r1','c3_r2','c3_r3'], axis=1, inplace=True)

In [848]:
df1

,Date,top3,up2
0,2024-04-16,598,79
1,2024-04-01,481,90
2,2024-03-16,626,78
3,2024-03-01,603,79
4,2024-02-16,395,43
...,...,...,...
695,1995-03-16,847,67
696,1995-03-01,454,71
697,1995-02-16,070,95
698,1995-02-01,162,48


In [849]:
window_length = 7
window_length

7

In [850]:
train = df.copy()
train.head((window_length+1))

,top3,up2
0,598,79
1,481,90
2,626,78
3,603,79
4,395,43
5,063,09
6,979,61
7,544,89


In [851]:
train_rows = train.values.shape[0]
train_samples = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
train_labels = np.empty([ train_rows - window_length, number_of_features], dtype=float)
for i in range(0, train_rows-window_length):
    train_samples[i] = train.iloc[i : i+window_length, 0 : number_of_features]
    train_labels[i] = train.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [852]:
train.values.shape[0]

700

In [853]:
train_samples[0]

array([[598.,  79.],
       [481.,  90.],
       [626.,  78.],
       [603.,  79.],
       [395.,  43.],
       [ 63.,   9.],
       [979.,  61.]])

In [854]:
train_labels[0]

array([544.,  89.])

In [855]:
scaler = StandardScaler()
transformed_dataset = scaler.fit_transform(train.values)
scaled_train_samples = pd.DataFrame(data=transformed_dataset, index=train.index)

In [856]:
scaled_train_samples.head(window_length+1)

,0,1
0,0.320595,0.936677
1,-0.092741,1.317782
2,0.419513,0.902031
3,0.338259,0.936677
4,-0.396560,-0.310576
5,-1.569446,-1.488537
6,1.666587,0.313051
7,0.129825,1.283137


In [857]:
x_train = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
y_train = np.empty([ train_rows - window_length, number_of_features], dtype=float)

for i in range(0, train_rows-window_length):
    x_train[i] = scaled_train_samples.iloc[i : i+window_length, 0 : number_of_features]
    y_train[i] = scaled_train_samples.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [858]:
y_train

array([[ 0.1298249 ,  1.28313651],
       [ 0.88230864,  1.14455282],
       [-1.44933114,  1.35242836],
       ...,
       [-1.5447164 ,  1.49101205],
       [-1.21969995, -0.13734634],
       [-0.42128996, -0.41451372]])

In [859]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mse

In [860]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a third Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))


In [861]:
model.compile(optimizer=Adam(learning_rate=0.005), loss ='mse', metrics=['accuracy'])

In [862]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, verbose=2)

Epoch 1/100
6/6 - 22s - loss: 1.0597 - accuracy: 0.4921 - 22s/epoch - 4s/step
Epoch 2/100
6/6 - 3s - loss: 1.0313 - accuracy: 0.4949 - 3s/epoch - 512ms/step
Epoch 3/100
6/6 - 4s - loss: 1.0199 - accuracy: 0.4574 - 4s/epoch - 638ms/step
Epoch 4/100
6/6 - 4s - loss: 1.0134 - accuracy: 0.5137 - 4s/epoch - 730ms/step
Epoch 5/100
6/6 - 3s - loss: 1.0031 - accuracy: 0.5195 - 3s/epoch - 513ms/step
Epoch 6/100
6/6 - 3s - loss: 1.0018 - accuracy: 0.5036 - 3s/epoch - 507ms/step
Epoch 7/100
6/6 - 3s - loss: 1.0053 - accuracy: 0.4834 - 3s/epoch - 571ms/step
Epoch 8/100
6/6 - 5s - loss: 0.9968 - accuracy: 0.4733 - 5s/epoch - 811ms/step
Epoch 9/100
6/6 - 3s - loss: 1.0040 - accuracy: 0.4993 - 3s/epoch - 506ms/step
Epoch 10/100
6/6 - 3s - loss: 0.9949 - accuracy: 0.5281 - 3s/epoch - 502ms/step
Epoch 11/100
6/6 - 3s - loss: 1.0006 - accuracy: 0.5065 - 3s/epoch - 519ms/step
Epoch 12/100
6/6 - 5s - loss: 0.9951 - accuracy: 0.4791 - 5s/epoch - 871ms/step
Epoch 13/100
6/6 - 3s - loss: 0.9913 - accuracy: 0

### Prediction without rounding up or down the results

In [863]:
print('-' * 40)
print('Prediction vs. GoundTruth without rounding up or down')
for i in range(1,10):
  test = df1.copy()
  test = test.tail((window_length+10-i))
  test = test.head((window_length+1))
  test_Date = df1.iloc[ (test.tail().index[-1]) ]['Date']
  test1 = test.head((window_length))
  test1.drop(['Date'], axis=1, inplace=True)
  test1 = np.array(test1)
  x_test = scaler.transform(test1)
  y_test_pred = model.predict(np.array([x_test]))
  y_test_true = test.drop(['Date'], axis=1, inplace=True)
  y_test_true = test.tail(1)
  print('Drawing  Date', test_Date)
  print('Prediction:\t', scaler.inverse_transform(y_test_pred).astype(int)[0])
  print('GoundTruth:\t', np.array(y_test_true)[0])
  print('-' * 40)

----------------------------------------
Prediction vs. GoundTruth without rounding up or down
1/1 [==============================] - 4s 4s/step
Drawing  Date 1995-05-17 00:00:00
Prediction:	 [375  56]
GoundTruth:	 ['381' '55']
----------------------------------------
1/1 [==============================] - 0s 42ms/step
Drawing  Date 1995-05-02 00:00:00
Prediction:	 [678  86]
GoundTruth:	 ['692' '88']
----------------------------------------
1/1 [==============================] - 0s 40ms/step
Drawing  Date 1995-04-16 00:00:00
Prediction:	 [547  95]
GoundTruth:	 ['581' '99']
----------------------------------------
1/1 [==============================] - 0s 42ms/step
Drawing  Date 1995-04-01 00:00:00
Prediction:	 [84  8]
GoundTruth:	 ['039' '06']
----------------------------------------
1/1 [==============================] - 0s 46ms/step
Drawing  Date 1995-03-16 00:00:00
Prediction:	 [811  68]
GoundTruth:	 ['847' '67']
----------------------------------------
1/1 [========================

### Prediction with rounding up the results

In [864]:
print('-' * 40)
print('Prediction vs. GoundTruth with rounding up')
for i in range(1,10):
  test = df1.copy()
  test = test.tail((window_length+10-i))
  test = test.head((window_length+1))
  test_Date = df1.iloc[ (test.tail().index[-1]) ]['Date']
  test1 = test.head((window_length))
  test1.drop(['Date'], axis=1, inplace=True)
  test1 = np.array(test1)
  x_test = scaler.transform(test1)
  y_test_pred = model.predict(np.array([x_test]))
  y_test_true = test.drop(['Date'], axis=1, inplace=True)
  y_test_true = test.tail(1)
  print('Drawing  Date', test_Date)
  print('Prediction:\t', scaler.inverse_transform(y_test_pred).astype(int)[0]+1)
  print('GoundTruth:\t', np.array(y_test_true)[0])
  print('-' * 40)

----------------------------------------
Prediction vs. GoundTruth with rounding up
1/1 [==============================] - 0s 40ms/step
Drawing  Date 1995-05-17 00:00:00
Prediction:	 [376  57]
GoundTruth:	 ['381' '55']
----------------------------------------
1/1 [==============================] - 0s 44ms/step
Drawing  Date 1995-05-02 00:00:00
Prediction:	 [679  87]
GoundTruth:	 ['692' '88']
----------------------------------------
1/1 [==============================] - 0s 58ms/step
Drawing  Date 1995-04-16 00:00:00
Prediction:	 [548  96]
GoundTruth:	 ['581' '99']
----------------------------------------
1/1 [==============================] - 0s 45ms/step
Drawing  Date 1995-04-01 00:00:00
Prediction:	 [85  9]
GoundTruth:	 ['039' '06']
----------------------------------------
1/1 [==============================] - 0s 40ms/step
Drawing  Date 1995-03-16 00:00:00
Prediction:	 [812  69]
GoundTruth:	 ['847' '67']
----------------------------------------
1/1 [==============================] -

### Predict the Future Drawing Results on June 1, 2o24

In [865]:
next_Date = '2024-05-02'
print('-' * 40)
print('Predict the Future Drawing on 4/16/2024')
next = df.copy()
next = next.tail((window_length))
next = np.array(next)
x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
print('Drawing  Date', next_Date)
print('Prediction without rounding up or down:\t', scaler.inverse_transform(y_next_pred).astype(int)[0])
print('Prediction with rounding up           :\t', scaler.inverse_transform(y_next_pred).astype(int)[0]+1)
print('Prediction with rounding down         :\t', scaler.inverse_transform(y_next_pred).astype(int)[0]-1)
print('-' * 40)

----------------------------------------
Predict the Future Drawing on 4/16/2024
1/1 [==============================] - 0s 38ms/step
Drawing  Date 2024-05-02
Prediction without rounding up or down:	 [450  58]
Prediction with rounding up           :	 [451  59]
Prediction with rounding down         :	 [449  57]
----------------------------------------


In [866]:
filtered_df = lt_2[lt_2['Date'] == next_Date]
g = filtered_df[['Date', 'top3', 'up2']]
g

,Date,top3,up2
1,2024-05-02,116,17


# แบบแยกหลัก

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
a = 1

In [ ]:
lt_3 = lt_2.iloc[a:sh[0], 0:sh[1]]

In [ ]:
dfp = lt_3.copy()

In [ ]:
df = dfp.reset_index()

In [ ]:
df

,index,Date,1st,top3,up2,weekday_dum,weekday,Year,Month,Day,Tens digit,Units digit,c3_r1,c3_r2,c3_r3
0,1,2024-05-02,980116,116,17,4,Wednesday,2024,5,2,1,7,1,1,6
1,2,2024-04-16,943598,598,79,3,Tuesday,2024,4,16,7,9,5,9,8
2,3,2024-04-01,803481,481,90,2,Monday,2024,4,1,9,0,4,8,1
3,4,2024-03-16,997626,626,78,7,Saturday,2024,3,16,7,8,6,2,6
4,5,2024-03-01,253603,603,79,6,Friday,2024,3,1,7,9,6,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,697,1995-03-16,646847,847,67,5,Thursday,1995,3,16,6,7,8,4,7
697,698,1995-03-01,411454,454,71,4,Wednesday,1995,3,1,7,1,4,5,4
698,699,1995-02-16,869070,070,95,5,Thursday,1995,2,16,9,5,0,7,0
699,700,1995-02-01,198162,162,48,4,Wednesday,1995,2,1,4,8,1,6,2


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
df2 = df.copy()

In [ ]:
df1 = df.copy()
df.drop(['index','Date','weekday_dum','weekday','Year','Month','Day','Tens digit','Units digit','c3_r1','c3_r2','c3_r3'], axis=1, inplace=True)

number_of_features = df.shape[1]

In [ ]:
df

,1st,top3,up2
0,980116,116,17
1,943598,598,79
2,803481,481,90
3,997626,626,78
4,253603,603,79
...,...,...,...
696,646847,847,67
697,411454,454,71
698,869070,070,95
699,198162,162,48


In [ ]:
df1.drop(['index','weekday_dum','weekday','Year','Month','Day','Tens digit','Units digit','c3_r1','c3_r2','c3_r3'], axis=1, inplace=True)

In [ ]:
df1

,Date,1st,top3,up2
0,2024-05-02,980116,116,17
1,2024-04-16,943598,598,79
2,2024-04-01,803481,481,90
3,2024-03-16,997626,626,78
4,2024-03-01,253603,603,79
...,...,...,...,...
696,1995-03-16,646847,847,67
697,1995-03-01,411454,454,71
698,1995-02-16,869070,070,95
699,1995-02-01,198162,162,48


In [ ]:
window_length = 7
window_length

7

In [ ]:
train = df.copy()
train.head((window_length+1))

,1st,top3,up2
0,980116,116,17
1,943598,598,79
2,803481,481,90
3,997626,626,78
4,253603,603,79
5,941395,395,43
6,607063,063,09
7,105979,979,61


In [ ]:
train_rows = train.values.shape[0]
train_samples = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
train_labels = np.empty([ train_rows - window_length, number_of_features], dtype=float)
for i in range(0, train_rows-window_length):
    train_samples[i] = train.iloc[i : i+window_length, 0 : number_of_features]
    train_labels[i] = train.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [ ]:
train.values.shape[0]

701

In [ ]:
train_samples[0]

array([[9.80116e+05, 1.16000e+02, 1.70000e+01],
       [9.43598e+05, 5.98000e+02, 7.90000e+01],
       [8.03481e+05, 4.81000e+02, 9.00000e+01],
       [9.97626e+05, 6.26000e+02, 7.80000e+01],
       [2.53603e+05, 6.03000e+02, 7.90000e+01],
       [9.41395e+05, 3.95000e+02, 4.30000e+01],
       [6.07063e+05, 6.30000e+01, 9.00000e+00]])

In [ ]:
train_labels[0]

array([1.05979e+05, 9.79000e+02, 6.10000e+01])

In [ ]:
scaler = StandardScaler()
transformed_dataset = scaler.fit_transform(train.values)
scaled_train_samples = pd.DataFrame(data=transformed_dataset, index=train.index)

In [ ]:
scaled_train_samples.head(window_length+1)

,0,1,2
0,1.638458,-1.379344,-1.209241
1,1.514524,0.322359,0.938094
2,1.039000,-0.090710,1.319073
3,1.697883,0.421213,0.903460
4,-0.827156,0.340011,0.938094
5,1.507048,-0.394333,-0.308745
6,0.372404,-1.566460,-1.486316
7,-1.328157,1.667480,0.314674


In [ ]:
x_train = np.empty([ train_rows - window_length, window_length, number_of_features], dtype=float)
y_train = np.empty([ train_rows - window_length, number_of_features], dtype=float)

for i in range(0, train_rows-window_length):
    x_train[i] = scaled_train_samples.iloc[i : i+window_length, 0 : number_of_features]
    y_train[i] = scaled_train_samples.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [ ]:
y_train

array([[-1.32815739,  1.66747992,  0.3146743 ],
       [ 0.43512398,  0.13171141,  1.28443834],
       [-0.47707586,  0.88370841,  1.14590062],
       ...,
       [ 1.26159392, -1.5417467 ,  1.49224492],
       [-1.01530998, -1.21694048, -0.13557329],
       [ 1.44254267, -0.41904696, -0.41264873]])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mse

In [ ]:
# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(240,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a third Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.005), loss ='mse', metrics=['accuracy'])

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, verbose=2)

Epoch 1/100
6/6 - 24s - loss: 1.0745 - accuracy: 0.3429 - 24s/epoch - 4s/step
Epoch 2/100
6/6 - 3s - loss: 1.0286 - accuracy: 0.3271 - 3s/epoch - 580ms/step
Epoch 3/100
6/6 - 3s - loss: 1.0105 - accuracy: 0.3516 - 3s/epoch - 520ms/step
Epoch 4/100
6/6 - 3s - loss: 1.0029 - accuracy: 0.3429 - 3s/epoch - 512ms/step
Epoch 5/100
6/6 - 4s - loss: 1.0014 - accuracy: 0.3242 - 4s/epoch - 741ms/step
Epoch 6/100
6/6 - 4s - loss: 0.9991 - accuracy: 0.3458 - 4s/epoch - 642ms/step
Epoch 7/100
6/6 - 3s - loss: 0.9938 - accuracy: 0.3357 - 3s/epoch - 514ms/step
Epoch 8/100
6/6 - 3s - loss: 0.9929 - accuracy: 0.3689 - 3s/epoch - 512ms/step
Epoch 9/100
6/6 - 4s - loss: 0.9905 - accuracy: 0.3818 - 4s/epoch - 667ms/step
Epoch 10/100
6/6 - 4s - loss: 0.9887 - accuracy: 0.3530 - 4s/epoch - 702ms/step
Epoch 11/100
6/6 - 3s - loss: 0.9932 - accuracy: 0.3617 - 3s/epoch - 513ms/step
Epoch 12/100
6/6 - 3s - loss: 1.0010 - accuracy: 0.3357 - 3s/epoch - 514ms/step
Epoch 13/100
6/6 - 4s - loss: 0.9801 - accuracy: 0

### Prediction without rounding up or down the results

In [ ]:
print('-' * 40)
print('Prediction vs. GoundTruth without rounding up or down')
for i in range(1,10):
  test = df1.copy()
  test = test.tail((window_length+10-i))
  test = test.head((window_length+1))
  test_Date = df1.iloc[ (test.tail().index[-1]) ]['Date']
  test1 = test.head((window_length))
  test1.drop(['Date'], axis=1, inplace=True)
  test1 = np.array(test1)
  x_test = scaler.transform(test1)
  y_test_pred = model.predict(np.array([x_test]))
  y_test_true = test.drop(['Date'], axis=1, inplace=True)
  y_test_true = test.tail(1)
  print('Drawing  Date', test_Date)
  print('Prediction:\t', scaler.inverse_transform(y_test_pred).astype(int)[0])
  print('GoundTruth:\t', np.array(y_test_true)[0])
  print('-' * 40)

----------------------------------------
Prediction vs. GoundTruth without rounding up or down
1/1 [==============================] - 3s 3s/step
Drawing  Date 1995-05-17 00:00:00
Prediction:	 [246024    381     56]
GoundTruth:	 [220381 '381' '55']
----------------------------------------
1/1 [==============================] - 0s 60ms/step
Drawing  Date 1995-05-02 00:00:00
Prediction:	 [495798    696     86]
GoundTruth:	 [504692 '692' '88']
----------------------------------------
1/1 [==============================] - 0s 62ms/step
Drawing  Date 1995-04-16 00:00:00
Prediction:	 [49570   543   100]
GoundTruth:	 [53581 '581' '99']
----------------------------------------
1/1 [==============================] - 0s 60ms/step
Drawing  Date 1995-04-01 00:00:00
Prediction:	 [451933     28      2]
GoundTruth:	 [427039 '039' '06']
----------------------------------------
1/1 [==============================] - 0s 59ms/step
Drawing  Date 1995-03-16 00:00:00
Prediction:	 [626287    810     66]
Gound

### Predict the Future Drawing Results on June 1, 2o24

In [ ]:
next_Date = '2024-05-16'
print('-' * 40)
print('Predict the Future Drawing on 4/16/2024')
next = df.copy()
next = next.tail((window_length))
next = np.array(next)
x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
print('Drawing  Date', next_Date)
print('Prediction without rounding up or down:\t', scaler.inverse_transform(y_next_pred).astype(int)[0])
print('Prediction with rounding up           :\t', scaler.inverse_transform(y_next_pred).astype(int)[0]+1)
print('Prediction with rounding down         :\t', scaler.inverse_transform(y_next_pred).astype(int)[0]-1)
print('-' * 40)

----------------------------------------
Predict the Future Drawing on 4/16/2024
1/1 [==============================] - 0s 36ms/step
Drawing  Date 2024-05-16
Prediction without rounding up or down:	 [295097    860     36]
Prediction with rounding up           :	 [295098    861     37]
Prediction with rounding down         :	 [295096    859     35]
----------------------------------------


In [ ]:
filtered_df = lt_2[lt_2['Date'] == next_Date]
filtered_df[['Date', '1st','top3', 'up2']]

,Date,1st,top3,up2
0,2024-05-16,205690,690,60
